In [ ]:
from IPython.display import HTML

# Multiclassification task on image data


Dataset: [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)

<img src="img/fashion-mnist-sprite.png">

In [ ]:
HTML('<img src="img/embedding.gif">')

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from IPython.display import SVG

from keras.models import Sequential
from keras.layers import InputLayer, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras_preprocessing.image import NumpyArrayIterator, ImageDataGenerator

np.random.seed(1234)
%matplotlib inline

## Load dataset

In [ ]:
fashion_mnist = fashion_mnist.load_data()

In [ ]:
(train_X, train_y), (test_X, test_y) = fashion_mnist

## Excersize: plot the data

In [ ]:
labels = np.unique(train_y)

plt.figure(figsize=(12, 6))

for i in labels:
    ax = plt.subplot(1, len(labels), i + 1)
    ax.axis('off')
    plt.imshow(train_X[train_y == i][0], 'gray')

## Scale data

In [ ]:
train_X = train_X.astype('float32') / 255.
test_X = test_X.astype('float32') / 255.

## Split data intro train, valid and test

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, stratify=train_y)


print('#train:', len(train_X), '#valid:', len(valid_X), '#test:', len(test_X))

## Reshape data

In [ ]:
train_X.shape

In [ ]:
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)
valid_X = valid_X.reshape(-1, 28, 28, 1)

## One-hot encode targets

In keras when we solve multiclassification task we apply one-hot encoding

In [ ]:
train_y_one_hot = to_categorical(train_y)
test_y_one_hot = to_categorical(test_y)
valid_y_one_hot = to_categorical(valid_y)

## Excersize: solve task with a simple fully connected network (like in regression notebook)

hints:
    
    * loss = 'categorical_crossentropy'
    * metrics = ['accuracy']

## Let's learn more about conv nets

[cs231n](http://cs231n.github.io/convolutional-networks)

## 1st cnn

In [ ]:
model = Sequential()

# Must define the input shape in the first layer of the neural network
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1))) 
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Take a look at the model summary
model.summary()

## Plot the model

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
!rm -rf /tmp/checkpoints
!mkdir /tmp/checkpoints

checkpointer = ModelCheckpoint('/tmp/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5')

history = model.fit(train_X, train_y_one_hot,
                    batch_size=64,
                    epochs=10,
                    validation_data=(valid_X, valid_y_one_hot),
                    callbacks=[checkpointer])

## Excersize: plot learning curves and choose the best epoch

## Excersize: change model structure (add more conv->max_pool blocks)

[how to do it](http://cs231n.github.io/convolutional-networks/#layerpat)